## Python常用语法糖

语法糖指代的是一些特殊的语法，它们又好看又好用，像糖一样。

常见的语法糖包括：

1. 匿名函数
2. 装饰器
3. 生成器
4. 上下文
5. 解包

### 课程目标

在本节课结束后，同学们会掌握：

1. Python常见语法糖的实际作用

### 匿名函数

在Python中，匿名函数，又称lambda函数，是一种无需显式定义、可以直接使用的函数。

In [1]:
def hi():
    a = 0
    a += 1
    print('hello world')
hi()

hwllo world


In [4]:
lambda x:x**2

<function __main__.<lambda>(x)>

关键字```lambda```声明我们要定义一个匿名函数，紧随```lambda```的则是我们要传入的变量，冒号后的值则是返回值。

In [5]:
f_add = lambda x,y:x+y
f_add(1,2)

3

匿名函数常用于“只会使用一次，但是必须定义成函数”的场合，如```map```。

map(func,iter)对于可迭代对象中的每个值，应用func并返回。

In [2]:
list(map(lambda x:x**2,[1,2,3]))

[1, 4, 9]

In [3]:
def sqr(x):
    return x**2
list(map(sqr,[1,2,3]))

[1, 4, 9]

### 装饰器

装饰器可以在不侵入函数内部代码的情况下，增强函数的功能。

In [4]:
from datetime import datetime
from functools import wraps

def log(level):
    def decorate(func):
        @wraps(func)
        def wrapper(*args, **kw):
            #------------------
            print(datetime.now(),f'Log level: {level}') # 打印当前时间
            #------------------
            return func(*args, **kw)
        return wrapper
    return decorate

@log(level='normal') # 用装饰器修饰函数
def hello():
    print('hello world')
    
hello()

2021-09-25 16:49:09.699262 Log level: normal
hello world


### 生成器

生成器是一个**可迭代对象**。当我们需要对一系列输入按顺序进行复杂处理时，而又不需要同时保存所有处理结果时，生成器可以显著减少资源开销。注意，生成器只能按顺序遍历，不能通过索引直接取到第若干个项。

In [15]:
(x**2 for x in [1,2,3])

<generator object <genexpr> at 0x7ff050245ac0>

In [5]:
g = (x**2 for x in [1,2,3])

for i in g:
    print(i)

1
4
9


我们也可以通过在函数中使用```yield```关键字来构造生成器。

In [6]:
def one_to_three():
    i = 0
    while i<3:
        i+=1
        yield i
        
    return
        
for i in one_to_three():
    print(i)

1
2
3


生成器也可以在迭代过程中接受外部输入，并且改变自己的行为。生成器未收到输入时会被挂起，不会消耗计算资源。生成器迭代完成后会抛出停止迭代的异常，需要自行处理。

In [7]:
def g():
    s = None
    while True:
        status = yield s
        if status==0:
            print('revieved 0, stop iteration')
            return
        elif status==1:
            print('recieved 1, yield 1')
            s = 1
        elif status==2:
            print('recieved 2, yield 2')
            s = 2

In [15]:
g()

<generator object g at 0x7fd46068ba50>

In [14]:
a.send(0)

revieved 0, stop iteration


StopIteration: 

In [9]:
a = g()
for i in [None,1,2,1,2,0]: # 注意：第一次必须发送None来初始化生成器
    print(a.send(i))

None
recieved 1, yield 1
1
recieved 2, yield 2
2
recieved 1, yield 1
1
recieved 2, yield 2
2
revieved 0, stop iteration


StopIteration: 

### 上下文

上下文允许我们定义进入```__enter__```和退出```__exit__```方法，开启上下文时会自动执行进入方法，离开上下文时会自动执行退出方法。在打开文件时经常会用到上下文管理，以避免因忘记关闭文件可能导致的文件句柄泄露。类似的，我们可以通过上下文管理来及时关闭不必要的数据库连接等。

In [1]:
with open('test.txt','w') as f:
    f.write('hello')
    
with open('test.txt','r') as f:
    print(f.readline())

hello


In [ ]:
f = open('test.txt','w')
f.write('hello')
f.close()

In [10]:
import pymysql

class DB:
    def __enter__(self):
        self.db = pymysql.connect(host='localhost',user='root',passwd='crawlme@2021FALL',db='mysql',charset='utf8')
        print('Entering context')
        return self.db # return的结果赋给as后面的变量

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.db.close()
        print('Exiting context')
        
create_schema_sql = '''CREATE DATABASE IF NOT EXISTS `markyfsun`;'''
        
create_table_sql = '''CREATE TABLE IF NOT EXISTS `lgulife_post`(
   `post_id` INT UNSIGNED,
   `post_title` VARCHAR(40) NOT NULL,
   `post_content` TEXT NOT NULL,
   PRIMARY KEY ( `post_id` )
)ENGINE=InnoDB DEFAULT CHARSET=utf8;'''

insert_data_sql = '''INSERT IGNORE INTO `lgulife_post` VALUES (0,'test','test');'''

with DB() as db: # 进入时自动连接数据库，退出时自动断开连接
    with db.cursor(pymysql.cursors.DictCursor) as c: # cursor也实现了上下文管理
        c.execute(create_schema_sql)
        c.execute('use markyfsun;')
        c.execute(create_table_sql)
        c.execute(insert_data_sql)
        db.commit()
        c.execute('select * from `lgulife_post`;')
        print(c.fetchone())


Entering context
{'post_id': 0, 'post_title': 'test', 'post_content': 'test'}
Exiting context


### 解包

Python支持```*```、```**```来将列表、字典等容器内的元素提取出来，使代码更简练。

In [16]:
a,b,c = [1,2,3]
print(a)

1


In [17]:
def f(*args, **kwargs):
    print(args)
    print(kwargs)

In [19]:
f(1,2,3,a='a',b='b')

(1, 2, 3)
{'a': 'a', 'b': 'b'}


In [ ]:
def add(x,y):
    return x+y

In [ ]:
f(1,2,3,a='a',b='b')

In [22]:
l = [1,2,3,4]
print(l)

[1, 2, 3, 4]


In [ ]:
print(l)
t = (1,*(2,3),4)
print(t)

In [24]:
d = dict(a='a',b='b',c='c')
print(d)

{'a': 'a', 'b': 'b', 'c': 'c'}


我们也可以利用解包同时完成多个赋值操作。

In [25]:
a,*b,c = [1,2,3,4,5]
print(a)
print(b)
print(c)

1
[2, 3, 4]
5


与解包相对应的是压包```zip```函数，它会将多个可迭代对象像拉链一样压成```tuple```。注意，```zip```返回的是一个生成器，只能顺序遍历。

In [29]:
for item in zip([1,2,3],['a','b','c']):
    print(item)

(1, 'a')
(2, 'b')
(3, 'c')


### 练习

定义一个支持上下文的类，对于进入上下文的返回值应用解包操作，解包后的元素中需要包含一个生成器，遍历该生成器并打印所有结果。定义一个装饰器用于修饰该类，使得在进入和退出上下文时可以打印出```"__enter__"```和```"__exit__"```以及对应时间。

In [ ]:
def log(level):
    def decorate(func):
        @wraps(func)
        def wrapper(*args, **kw):
            #------------------
            print(datetime.now(),f'Log level: {level}') # 打印当前时间
            #------------------
            return func(*args, **kw)
        return wrapper
    return decorate

In [38]:
class DB:
    @log(level='normal')
    def __enter__(self):
        return (x**2 for x in [1,2,3]),1,2
    @log(level='__exit__')
    def __exit__(self, exc_type, exc_val, exc_tb):
        pass
        

In [39]:
with DB() as db:
    print('in context')
    a,b,c = db
    for i in a:
        print(i)

2021-09-25 17:30:18.289625 Log level: normal
in context
1
4
9
2021-09-25 17:30:18.290094 Log level: __exit__
